In [ ]:
import time
import csv
import random
def measure_data(reltime):
    """
    Simulates a sensor measurement and writes the data to a CSV file.
    Parameters:
        reltime (float): The relative time (in seconds) since the measurement started.
    This function simulates obtaining a measurement (for example, using a random value)
    and appends the relative time and the measurement value to a CSV file.
    """
    try:
        # Simulate a sensor measurement (random value between 0 and 100)
        measurement = random.uniform(0, 100)
        # Append the measurement data to a CSV file
        with open('measurement_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([reltime, measurement])
    except Exception as e:
        print(f"Error in measure_data: {e}")
        
def time_provider(sampling_rate=512):
    """
    Generator function that yields relative time values at the specified sampling rate.
    Parameters:
        sampling_rate (int): Number of samples per second (e.g., 512 or 1024).
    Yields:
        float: The relative time (in seconds) since measurement start.
    """
    interval = 1.0 / sampling_rate  # Time interval between samples in seconds
    start_time = time.time()         # Record the start time
    next_sample_time = start_time    # Initialize the next sample time
    while True:
        current_time = time.time()
        # Wait until the next scheduled sample time is reached
        if current_time < next_sample_time:
            time.sleep(next_sample_time - current_time)
        # Calculate the relative time from the start
        reltime = next_sample_time - start_time
        yield reltime
        # Update the next sample time for the following iteration
        next_sample_time += interval

def continuous_measurement(sampling_rate=512):
    """
    Continuously obtains relative time values from the time_provider and passes them
    to the measure_data function for processing. The loop runs indefinitely.
    Parameters:
        sampling_rate (int): The number of samples per second (e.g., 512 or 1024).
    """
    for reltime in time_provider(sampling_rate):
        try:
            # Pass the relative time to the measurement function
            measure_data(reltime)
        except Exception as e:
            print(f"Error in continuous_measurement: {e}")
if __name__ == '__main__':
    # Start continuous measurement at a sampling rate of 512 samples per second.
    continuous_measurement(sampling_rate=512)






In [6]:
import time
def time_provider(sampling_rate=512):
    """
    Generator function that yields relative time values at the specified sampling rate.
    Parameters:
        sampling_rate (int): Number of samples per second (e.g., 512 or 1024).
    Yields:
        float: The relative time (in seconds) since measurement start.
    """
    interval = 1.0 / sampling_rate  # Time interval between samples in seconds
    start_time = time.time()         # Record the start time
    next_sample_time = start_time    # Initialize the next sample time
    while True:
        current_time = time.time()
        # Wait until the next scheduled sample time is reached
        if current_time < next_sample_time:
            time.sleep(next_sample_time - current_time)
        # Calculate the relative time from the start
        reltime = next_sample_time - start_time
        yield reltime
        # Update the next sample time for the following iteration
        next_sample_time += interval

In [ ]:
print(time_provider(sampling_rate=512))
from itertools import islice # Collect a fixed number of samples (sampling_rate) 
import numpy as np
time_samples = np.array(list(islice(time_provider(sampling_rate=512),512)))
print(time_samples)

<generator object time_provider at 0x000001DE0003D0C8>


In [1]:
import threading
import time
import lib_KT2700
import lib_NIDAQ


def daq_KT2700():
    # KT2700
    rm, rl = lib_KT2700.query()
    KT2700 = dict()
    KT2700["save_path"] = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"
    KT2700["columnnames"] = ["reltime", "Voltage"]
    KT2700["filename"] = lib_KT2700.create_csv(
        KT2700["save_path"], KT2700["columnnames"]
    )
    KT2700["instrument"] = rm.open_resource(rl[3])
    lib_KT2700.initialize(instrument=KT2700["instrument"])
    lib_KT2700.DataAcquisition(
        KT2700["filename"],
        KT2700["save_path"],
        KT2700["columnnames"],
        refresh_rate=0.01,
        instrument=KT2700["instrument"],
    )


def daq_NI9215():
    # NIDAQ
    NI9215 = dict()
    NI9215["save_path"] = r"C:/Users/IPMU/Desktop/2025-gripper-run3/NIDAQ/"
    NI9215["columnnames"] = [
        "Reltime",
        "Current A",
        "Current B",
        "Voltage A",
        "Voltage B",
    ]
    NI9215["filename"] = lib_NIDAQ.create_csv(
        NI9215["save_path"], NI9215["columnnames"]
    )
    lib_NIDAQ.DataAcquisition(
        NI9215["save_path"],
        NI9215["filename"],
        NI9215["columnnames"],
        sampling_rate=512,
    )


if __name__ == "__main__":
    # Create threads for each DAQ function
    t1 = threading.Thread(target=daq_KT2700, name="KT2700", daemon=True)
    t2 = threading.Thread(target=daq_NI9215, name="NI9215", daemon=True)
    # Start the threads
    t1.start()
    t2.start()
    try:
        # Keep the main thread alive to let the DAQ threads run
        while True:
            time.sleep(5)
    except KeyboardInterrupt:
        print("Stopping threads...")
        # With daemon threads, they will exit when the main thread ends.


Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/NIDAQ/20250221145709_NIDAQ_NI9215.csv
resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')
Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40

Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221145709_MULTIMETER_KT2700.csv
+3.67033863E-05VDC,+39.476SECS,+00291RDNG#

KT2700 initialization complete.
Stopping threads...
